In [1]:
#export
from k1lib.callbacks import Callback, Cbs
import k1lib; from k1lib import cli

In [2]:
#export
_li = 30
class IOData:
    def __init__(self, ioProfiler, mS:k1lib.selector.ModuleSelector):
        self.ioProfiler = ioProfiler; self.mS = mS
        self.iS = None; self.oS = None
        self.handle = None; self.hook()
    def hook(self):
        def hk(m, i, o):
            self.iS = list(k1lib.squeeze(i, True).shape)
            self.oS = list(k1lib.squeeze(o, True).shape)
        self.handle = self.mS.nn.register_forward_hook(hk)
    def unhook(self): self.handle.remove()
    def __getstate__(self):
        answer = dict(self.__dict__)
        del answer["mS"]; del answer["ioProfiler"]; return answer
    def __setstate__(self, state): self.__dict__.update(dict(state))
    def __str__(self):
        a = f"{self.iS}".ljust(_li); b = f"{self.oS}".ljust(_li)
        return f"{a}{b}"

In [3]:
#export
class IOProfiler(Callback):
    """Gets input and output shapes of each layer.
Example::
    
    l = k1lib.Learner.sample()
    l.cbs.add(Cbs.Profiler())
    # views table
    l.Profiler.io
    # views table highlighted
    l.Profiler.io.css("#lin1")
"""
    def startRun(self):
        if not hasattr(self, "selector"): # if no selectors found
            self.selector = self.l.model.select("")
        for m in self.selector.modules(): m.data = IOData(self, m)
        self.selector.displayF = lambda m: (k1lib.fmt.txt.red if "_ioProf_" in m else k1lib.fmt.txt.identity)(m.data)
    def startStep(self): return True
    def _run(self):
        """Runs everything"""
        with self.cbs.suspendEval(): self.l.run(1, 1)
        for m in self.selector.modules(): m.data.unhook()
    def css(self, css:str):
        """Selects a small part of the network to highlight. See also: :mod:`k1lib.selector`."""
        self.selector.parse(k1lib.selector.preprocess(css, "_ioProf_"))
        print(self.__repr__()); self.selector.clearProps()
    def __repr__(self):
        header = "input shape".ljust(_li) + "output shape".ljust(_li)
        c = self.selector.__repr__(intro=False, header=header).split("\n") | cli.tab() | cli.join("\n")
        return f"""IOProfiler:\n{c}

Can...
- iop.css("..."): highlights a particular part of the network
- iop.selector: to get internal k1lib.ModuleSelector object"""

In [4]:
l = k1lib.Learner.sample()
l.cbs.add(Cbs.Profiler())
l.Profiler.io.css("#lin1")

Epoch cancelled: Batch 1 reached.
IOProfiler:
                                input shape                   output shape                  
    root: Model                 [32]                          [32]                          
        lin1: LinBlock          [32, 1]                       [32, 3]                       
            lin: Linear         [32, 1]                       [32, 3]                       
            relu: ReLU          [32, 3]                       [32, 3]                       
        lin2: Linear            [32, 3]                       [32]                          

Can...
- iop.css("..."): highlights a particular part of the network
- iop.selector: to get internal k1lib.ModuleSelector object


In [5]:
!../../../export.py callbacks/profilers/io

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.15
Uninstalling k1lib-0.15:
  Successfully uninstalled k1lib-0.15
running install
/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-